In [1]:
!cp -r /kaggle/input/doubleroc/* /kaggle/working/SASV_DoubleRoc-main

In [2]:
!ln -s /kaggle/input/nndang-fusion-test /kaggle/working/SASV_DoubleRoc-main/fusion_test

In [3]:
!ln -s /kaggle/input/nndang-spoofing-data /kaggle/working/SASV_DoubleRoc-main/spoofing-data

In [4]:
!ln -s /kaggle/input/nndang-vn-celeb/VLSP23_VN_celeb2 /kaggle/working/SASV_DoubleRoc-main/vn-celeb

In [5]:
!ln -s /kaggle/input/weight-cm /kaggle/working/SASV_DoubleRoc-main/CM_weight

In [6]:
!ln -s /kaggle/input/tst-enr /kaggle/working/SASV_DoubleRoc-main/enrollment_txt

In [7]:
!ln -s /kaggle/input/ecapa-tdnn /kaggle/working/SASV_DoubleRoc-main/ecapa-tdnn

In [8]:
!ln -s /kaggle/input/vn-celeb-txt-final /kaggle/working/SASV_DoubleRoc-main/vn_celeb_txt

In [9]:
!ln -s /kaggle/input/emb-roc /kaggle/working/SASV_DoubleRoc-main/emb_roc

In [10]:
!ln -s /kaggle/input/public-tst /kaggle/working/SASV_DoubleRoc-main/public_tst

In [11]:
!ln -s /kaggle/input/ecapa-weight /kaggle/working/SASV_DoubleRoc-main/weight_ecapa

In [12]:
!ln -s /kaggle/input/cm-weight-new /kaggle/working/SASV_DoubleRoc-main/cm_weight_new

In [13]:
!ln -s /kaggle/input/pkl-s2pecnet-data /kaggle/working/SASV_DoubleRoc-main/pkl-s2pecnet-data

In [14]:
!ln -s /kaggle/input/public-tst-pkl /kaggle/working/SASV_DoubleRoc-main/pubilc_tst_pkl

In [15]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py
import os
import tqdm
import torch
import argparse
import numpy as np
import pickle as pk
import torch.nn as nn
from typing import Dict, List
import torch.nn.functional as F
from torch.utils.data import DataLoader
from test_dataset import get_dataloader
from models.ResNet34V2 import MainModel as ResNet34V2
from models.ECAPATDNN import MainModel as ECAPATDNN
from models.Res2Net50V2 import MainModel as Res2Net50V2
from models.Res2NeXt50 import MainModel as Res2NeXt50
from models.WavLMLarge import MainModel as WavLMLarge

vn_celeb_trn_path = "/kaggle/working/SASV_DoubleRoc-main/vn_celeb_txt/train_list_final.txt"
vn_celeb_tst_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
cm_tst_bf = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
 
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--LA_dataset_path', type=str, default='LA', help='The model to Run')
    parser.add_argument(
        "--ASV_embedings_save_path", type=str, default="embeddings"
    )
    parser.add_argument(
        "--model", type=str, default="ECAPATDNN"
    )
    parser.add_argument(
        "--gpu", type=str, default="0"
    )
    return parser.parse_args()


if __name__ == '__main__':
    args = get_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Device: {}".format(device))
    LA_dataset_path = args.LA_dataset_path
    if not os.path.exists(LA_dataset_path):
        print('LA Datset is not exist.')
        exit(0)
    if args.model == 'ECAPATDNN':
        model = ECAPATDNN('/kaggle/working/SASV_DoubleRoc-main/weight_ecapa/model_0070.model')
    elif args.model == 'ResNet34V2':
        model = ResNet34V2('./weights/resnet34v2.model')
    elif args.model == 'Res2Net50V2':
        model = Res2Net50V2('./weights/res2net50v2.model')
    elif args.model == 'Res2NeXt50':
        model = Res2NeXt50('./weights/res2next50.model')
    elif args.model == 'WavLMLarge':
        model = WavLMLarge('./weights/wavlm_large_finetune.model')
    else:
        print(f'model {args.model} is not supoort.')
        exit(0)
    model = model.to(device)
    model = model.eval()
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    #for data_dirname in ['train_celeb', 'valid_celeb', 'enr_set']:
    #if data_dirname == "train_celeb":
    #    meta_lines = open(vn_celeb_trn_path, "r").readlines()
    #if data_dirname == "valid_celeb":
    meta_lines = open(vn_celeb_tst_path, "r").readlines()
    #elif data_dirname == "enr_set":
    utt_enr_list = []
    utt_tst_list = []
    for line in meta_lines:
        tmp = line.strip().split('\t')
        spk = tmp[0]
        utt = tmp[1]
        if utt not in utt_tst_list:
            utt_tst_list.append(utt)
        if spk not in utt_enr_list:
            utt_enr_list.append(spk)
    path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/"
    sasv_tst = get_dataloader(path, utt_tst_list)
    sasv_enr = get_dataloader(path, utt_enr_list)
    #for sasv_dataloader in ["sasv_tst", "sasv_enr"]:
    spk_emb_dic = {}
    with torch.no_grad():
        for idx, (data_1s, data_2s, files) in  tqdm.tqdm(enumerate(sasv_tst), total = len(sasv_tst)):
            data_1 = data_1s[0].cuda()
            data_2 = data_2s[0].cuda()
            file = files[0]

            embedding_1 = model(data_1)
            embedding_1_norm = F.normalize(embedding_1, p=2, dim=1)
            embedding_2 = model(data_2)
            embedding_2_norm  = F.normalize(embedding_2, p=2, dim=1)
            embedding = torch.cat([embedding_1, embedding_2], dim=0).detach().cpu().numpy()
            embedding_norm = torch.cat([embedding_1_norm, embedding_2_norm], dim=0).detach().cpu().numpy()
            spk_emb_dic[file] = embedding_norm
            #spk_emb_norm_dic[file] = embedding_norm
    os.makedirs(os.path.join(args.ASV_embedings_save_path, args.model), exist_ok=True)
    with open( f"{os.path.join(args.ASV_embedings_save_path, args.model)}/sasv_tst.pk", "wb") as f:
        pk.dump(spk_emb_dic, f)
        
    spk_emb_dic = {}
    with torch.no_grad():
        for idx, (data_1s, data_2s, files) in  tqdm.tqdm(enumerate(sasv_enr), total = len(sasv_enr)):
            data_1 = data_1s[0].cuda()
            data_2 = data_2s[0].cuda()
            file = files[0]

            embedding_1 = model(data_1)
            embedding_1_norm = F.normalize(embedding_1, p=2, dim=1)
            embedding_2 = model(data_2)
            embedding_2_norm  = F.normalize(embedding_2, p=2, dim=1)
            embedding = torch.cat([embedding_1, embedding_2], dim=0).detach().cpu().numpy()
            embedding_norm = torch.cat([embedding_1_norm, embedding_2_norm], dim=0).detach().cpu().numpy()
            spk_emb_dic[file] = embedding_norm
            #spk_emb_norm_dic[file] = embedding_norm   
    with open( f"{os.path.join(args.ASV_embedings_save_path, args.model)}/sasv_enr.pk", "wb") as f:
        pk.dump(spk_emb_dic, f)



Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py


In [16]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/test_dataset.py
import os
import torch
import numpy as np
from glob import glob
import soundfile as sf
from torch.utils.data import Dataset, DataLoader

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

def transform_string(dong):
    # Sử dụng biểu thức chính quy để tìm các phần tử id trong dòng
    match = re.match(r'BF_id(\d{5})_(\d{8})', dong)
    
    if match:
        # Lấy các phần tử id từ kết quả tìm được
        id1, id2 = match.groups()
        # Chuyển đổi sang định dạng mong muốn
        ket_qua = f'id{id1.zfill(5)}/{id2.zfill(8)}'
        return ket_qua
    else:
        # Trả về dòng ban đầu nếu không tìm thấy kết quả phù hợp
        return dong


class SASVTestDataset(Dataset):
    def __init__(self, eval_path, list_id, num_frames=300):
        super().__init__()
        self.eval_path = eval_path
        self.list_id = list_id
        self.num_frames = num_frames
    
    def __len__(self):
        return len(self.list_id)
    
    def __getitem__(self, index):
        # return super().__getitem__(index)
        file = self.list_id[index]
        audio_path = self.eval_path + f"{file}"
        audio, _  = sf.read(audio_path)
        # Full utterance 完整的一句
        data_1 = torch.FloatTensor(np.stack([audio],axis=0))

        # Spliited utterance matrix 将语音切片
        max_audio = self.num_frames * 160 + 240
        if audio.shape[0] <= max_audio:
            shortage = max_audio - audio.shape[0]
            audio = np.pad(audio, (0, shortage), 'wrap')
        feats = []
        startframe = np.linspace(0, audio.shape[0]-max_audio, num=5)
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])
        feats = np.stack(feats, axis = 0).astype(np.float)
        data_2 = torch.FloatTensor(feats)
        return data_1, data_2, file

def get_dataloader(eval_path, list_id):
    sasv_dataset = SASVTestDataset(eval_path, list_id)
    sasv_dataloader = DataLoader(dataset=sasv_dataset, batch_size=1, num_workers=4)
    return sasv_dataloader





Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/test_dataset.py


In [17]:
!python /kaggle/working/SASV_DoubleRoc-main/ASV/save_embeddings.py --model ECAPATDNN --LA_dataset_path "/kaggle/working/SASV_DoubleRoc-main/public_tst/"

Device: cuda
spk model load model_0070.model
100%|███████████████████████████████████████| 9063/9063 [03:21<00:00, 44.93it/s]


In [18]:
%%writefile /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py
import os
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
import soundfile as sf
from torch import Tensor
import torch, time, sys, tqdm
from utils import get_all_EERs
from models.AASIST import MainModel as CMModel
from models.S2pecNet import MainModel as S2pecNet
from torch.utils.data import Dataset, DataLoader

def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--LA_dataset_path', type=str, default='/kaggle/working/SASV_DoubleRoc-main/public_tst/', help='The model to Run')
    parser.add_argument(
        "--ASV_embedings_save_path", type=str, default="/kaggle/working/embeddings/ECAPATDNN/"
    )
    parser.add_argument(
        "--model", type=str, default="ECAPATDNN"
    )
    parser.add_argument(
        "--norm", type=str, default="l2norm"
    )
    parser.add_argument(
        "--gpu", type=str, default="0"
    )
    return parser.parse_args()

def init_cm_model(cm_weight_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    cm_model = CMModel(cm_weight_path)
    cm_model = cm_model.to(device)
    cm_model = cm_model.eval()
    for module in cm_model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    return cm_model

def init_cm_model_1(cm_weight_path):
    device = "cuda" #if torch.cuda.is_available() else "cpu"
    cm_model = S2pecNet(cm_weight_path)
    cm_model = cm_model.to(device)
    cm_model = cm_model.eval()
    for module in cm_model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.track_running_stats = False
            module.eval()
        elif isinstance(module, nn.BatchNorm1d):
            module.track_running_stats = False
            module.eval()
    return cm_model

def transform_string(input_string):
    # Tách lấy phần cuối của đường dẫn (tức là tên tệp tin)
    file_name = os.path.basename(input_string)
    
    # Loại bỏ phần mở rộng '.wav' và thay thế bằng 'LFCC.pkl'
    transformed_name = file_name.replace('.wav', 'LFCC.pkl')
    
    return transformed_name

def repeat_padding(spec, ref_len):
    mul = int(np.ceil(ref_len / spec.shape[1]))
    spec = spec.repeat(1, mul)[:, :ref_len]
    return spec

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x

class Dataset_devNeval(Dataset):
    def __init__(self, base_dir, items, enr_emb, asv_embd, pkl_list, pkl_dir):
        self.base_dir = base_dir
        self.items = items
        self.cut = 64600  # take ~4 sec audio (64600 samples), 这个对于CM系统可能需要修改
        self.asv_embd = asv_embd
        self.enr_emb = enr_emb
        self.pkl_list = pkl_list
        self.pkl_dir = pkl_dir
        for key in asv_embd:
            asv_embd[key] = asv_embd[key].mean(0)
        for key in enr_emb:
            enr_emb[key] = enr_emb[key].mean(0)
        
    def __len__(self):
        return len(self.items)

    def __getitem__(self, index):
        spk, tst = self.items[index]
        X, _ = sf.read('{}{}'.format(self.base_dir, tst))
        pkl_id = transform_string(tst)
        with open(self.pkl_list[pkl_id], 'rb') as f:
            lfcc = pickle.load(f)
        lfcc = torch.from_numpy(lfcc)
        this_feat_len = lfcc.shape[1]
        if this_feat_len > 750:
            lfcc = lfcc[:, :750]
            
        if this_feat_len < 750:
            lfcc = repeat_padding(lfcc, 750)
        lfcc = lfcc.float()
        X_pad = pad(X, self.cut)
        x_inp = Tensor(X_pad)
        return self.enr_emb[spk], self.asv_embd[tst], x_inp, lfcc, index

def gen_dev_and_eval_item_list(meta_path):
    items = []
    with open(meta_path, 'r', encoding='utf-8') as fin:
        for line in fin:
            tmp = line.strip().split('\t')
            enr = tmp[0]
            tst = tmp[1]
            items.append([enr, tst])
    return items


def gen_loader(database_path, embedding_path, batch_size, n_cpu, norm):

    dev_database_path  = database_path + "/ASVspoof2019_LA_dev/"
    eval_database_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/"

    dev_trial_path  = database_path + "/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.gi.trl.txt"
    eval_trial_path = "/kaggle/working/SASV_DoubleRoc-main/public_tst/example_submission/submission.txt"
    eval_pk_path = "/kaggle/working/SASV_DoubleRoc-main/pubilc_tst_pkl/PKL_public_test/eval/"
    # dev dataset setting
    # dev_spk2ids, dev_items = gen_dev_and_eval_item_list([database_path + '/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.male.trn.txt',
    #                                                      database_path + '/ASVspoof2019_LA_asv_protocols/ASVspoof2019.LA.asv.dev.female.trn.txt'],
    #                                                     dev_trial_path)
    # dev_asv_embd = pickle.load(open(os.path.join(embedding_path, f'ASVspoof2019_LA_dev_{norm}.pk'), 'rb'))

    # dev_dataset = Dataset_devNeval(base_dir=dev_database_path, spk2ids=dev_spk2ids, items=dev_items, asv_embd=dev_asv_embd)
    # dev_loader = DataLoader(dataset=dev_dataset,
    #                         batch_size=batch_size,
    #                         shuffle=False,
    #                         drop_last=False,
    #                         pin_memory=True,
    #                         num_workers=n_cpu)
    # eval dataset setting
    eval_items = gen_dev_and_eval_item_list(eval_trial_path)
    eval_asv_embd = pickle.load(open("/kaggle/working/embeddings/ECAPATDNN/sasv_tst.pk",'rb'))
    enr_emb = pickle.load(open("/kaggle/working/embeddings/ECAPATDNN/sasv_enr.pk",'rb'))
    
    public_tst_pkl_list = {}
    files = os.listdir(eval_pk_path)
    for file in files:
        if os.path.isfile(os.path.join(eval_pk_path, file)):
            public_tst_pkl_list[file] = os.path.join(eval_pk_path, file)
        
    eval_dataset = Dataset_devNeval(base_dir=eval_database_path, items=eval_items, enr_emb = enr_emb, asv_embd=eval_asv_embd, pkl_list = public_tst_pkl_list, pkl_dir = eval_pk_path)
    eval_loader = DataLoader(dataset=eval_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             drop_last=False,
                             pin_memory=True,
                             num_workers=n_cpu)
    return eval_loader

if __name__ == '__main__':
    args = get_args()
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
    device = "cuda" #if torch.cuda.is_available() else "cpu"
    print("Device: {}".format(device))
    print(f'Model: {args.model}')
    print(f'Norm: {args.norm}')
    assert args.norm in ['l2norm', 'nonorm']
    LA_dataset_path = args.LA_dataset_path
    ASV_embedings_save_path = args.ASV_embedings_save_path
    cm_model = init_cm_model_1('/kaggle/working/SASV_DoubleRoc-main/cm_weight_new/epoch_15_0.134.pth')
    cos = nn.CosineSimilarity(dim=1, eps=1e-6).to(device)
    eval_loader = gen_loader(LA_dataset_path, os.path.join(ASV_embedings_save_path, args.model), batch_size=64, n_cpu=4, norm=args.norm)
    tag_2_loader = {'eval': eval_loader}
    for tag in tag_2_loader.keys():
        loader = tag_2_loader[tag]
        id2score = {}
        id2label = {}
        scores_list, labels_list = [], []
        score_list_1 = []
        for idx, (asv_embd_enr, asv_embd_tst, fusion_cm_data, lfcc, index) in enumerate(loader):
            with torch.no_grad():
                output = cm_model.forward(fusion_cm_data.to(device), lfcc.to(device))[1]
                score1 =  output.softmax(-1).detach().cpu().numpy()[:, 1]
                #score_1_list.append(score1)
                score2 = cos(asv_embd_enr.to(device), asv_embd_tst.to(device)).detach().cpu().numpy()
                score2 = (score2 + 1) / 2 #rút về khoảng [0,1]
                score = score1 * score2
                score_1 = score2 - score1
                score_1 = abs(score_1.mean())
                score_list_1.append(score_1)
                index = index.detach().cpu().numpy()

                for i, s in zip(index, score):
                    #id2label[i] = l
                    id2score[i] = s
        for i in range(len(id2score.keys())):
            scores_list.append(id2score[i])
            #labels_list.append(id2label[i])
        #sasv_EER, sv_EER, spf_EER = get_all_EERs(preds=scores_list, keys=labels_list)
        #print("%s_sasvEER %2.2f%%, eval_svEER %2.2f%%, eval_spfEER %2.2f%%" % (tag, sasv_EER * 100, sv_EER * 100, spf_EER * 100))
with open("/kaggle/working/SASV_DoubleRoc-main/public_tst_result_mul.txt", 'w') as f:
    for item in scores_list:
        f.write("%s\n" %item)

with open("/kaggle/working/SASV_DoubleRoc-main/public_tst_result_plus.txt", 'w') as f:
    for item in score_list_1:
        f.write("%s\n" %item)


Overwriting /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py


In [19]:
!python /kaggle/working/SASV_DoubleRoc-main/ASV/test_asv_cm.py --norm l2norm --model ECAPATDNN 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Device: cuda
Model: ECAPATDNN
Norm: l2norm
cm model load epoch_15_0.134.pth
